In [11]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv() 

True

## Cycle Graph Generation

In [12]:
from chatsky_llm_autoconfig.algorithms.dialogue_augmentation import DialogAugmentation
from chatsky_llm_autoconfig.algorithms.dialogue_generation import DialogueSampler
from chatsky_llm_autoconfig.algorithms.topic_graph_generation import CycleGraphGenerator
from chatsky_llm_autoconfig.metrics.llm_metrics import are_triplets_valid
from typing import List, Dict, Any
import os
import json
from datetime import datetime
from pathlib import Path

def generate_dialogues(
    topics: List[str], 
    num_augmentations: int = 2,
    model_name: str = "gpt-4o-mini"
) -> List[Dict[str, Any]]:
    """
    Collects dialogues for each topic and returns them in a structured format.
    
    Args:
        topics: List of conversation topics
        num_augmentations: Number of augmented dialogues to generate per topic
        model_name: Name of the model to use
        
    Returns:
        List of dictionaries containing graph, topic, and dialogues for each topic
    """
    # Initialize components
    graph_generator = CycleGraphGenerator()
    sampler = DialogueSampler()
    augmenter = DialogAugmentation()
    
    model = ChatOpenAI(
        model=model_name, 
        api_key=os.getenv("OPENAI_API_KEY"), 
        base_url=os.getenv("OPENAI_BASE_URL"), 
        temperature=0
    )
    
    results = []
    
    for topic in topics:
        # Generate graph and validate
        graph = graph_generator.invoke(topic=topic)
        is_valid = are_triplets_valid(graph, model, topic)
        
        if is_valid["value"]:
            print(f"Graph: {graph}")
            sampled_dialogue = sampler.invoke(graph, 1, -1)[0]
            
            print(f"Sampled dialogue: {sampled_dialogue}")
            
            # Initialize dialogues array with the sampled dialogue
            dialogues = [sampled_dialogue.model_dump()]
            
            # Generate augmented versions
            for _ in range(num_augmentations):
                augmented = augmenter.invoke(dialogue=sampled_dialogue, topic=topic)
                dialogues.append(augmented.model_dump())
            
            # Create result entry
            result_entry = {
                "graph": graph.graph_dict,
                "topic": topic,
                "dialogues": dialogues
            }
            
            results.append(result_entry)
            
    return results

def save_dataset(data: List[Dict[str, Any]], num_augmentations: int):
    """
    Saves the dataset with a structured filename.
    
    Args:
        data: List of dictionaries containing graph, topic, and dialogues
        num_augmentations: Number of augmented dialogues generated
    """
    # Create dataset directory if it doesn't exist
    dataset_dir = Path("./dataset")
    dataset_dir.mkdir(parents=True, exist_ok=True)
    
    # Generate filename with timestamp and dataset info
    total_dialogues = 1 + num_augmentations
    filename = f"dialogue_dataset_{len(data)}_topics_{total_dialogues}.json"
    
    # Full path for the file
    file_path = dataset_dir / filename
    
    # Convert Graph objects to their dictionary representation
    serializable_data = []
    for entry in data:
        serializable_entry = {
            "graph": entry["graph"],  # Use graph_dict instead of Graph object
            "topic": entry["topic"],
            "dialogues": entry["dialogues"]
        }
        serializable_data.append(serializable_entry)
    
    # Save the data
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(serializable_data, f, indent=2, ensure_ascii=False)
    
    print(f"Dataset saved to: {file_path}")
    return file_path
# Example usage
topics = [
    "booking a hotel room",                    # Travel/Accommodation
    "scheduling a doctor's appointment",       # Healthcare
    "ordering food delivery",                  # Food Service
    "tech support for a laptop issue",         # Technical Support
    "buying movie tickets",                    # Entertainment
    "gym membership registration",             # Fitness
    "car maintenance service",                 # Automotive
    "flight reservation",                      # Travel
    "pizza delivery customization",            # Food Service
    "hair salon appointment booking",          # Personal Care
    "internet service installation",           # Utilities
    "banking account assistance",              # Financial Services
    "pet grooming appointment",               # Pet Services
    "mobile phone plan upgrade",              # Telecommunications
    "house cleaning service booking",          # Home Services
    "online therapy session booking",          # Mental Health
    "language course enrollment",              # Education
    "covid-19 test scheduling",               # Healthcare
    "food delivery subscription",              # Food Service
    "remote work tech support"                # Technical Support
]

# Number of augmentations (total dialogues will be num_augmentations + 1)
num_augmentations = 2

# Collect dialogues
dialogue_collection = generate_dialogues(topics, num_augmentations=num_augmentations)

saved_path = save_dataset(dialogue_collection, num_augmentations)


INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"


Graph: graph_dict={'edges': [{'source': 1, 'target': 2, 'utterances': ["I'd like to book a hotel room"]}, {'source': 2, 'target': 3, 'utterances': ['A double room for two nights']}, {'source': 3, 'target': 4, 'utterances': ["Yes, that's correct"]}, {'source': 4, 'target': 5, 'utterances': ["Here's my credit card information"]}, {'source': 5, 'target': 2, 'utterances': ["I'd like to book another room"]}], 'nodes': [{'id': 1, 'label': 'welcome', 'is_start': True, 'utterances': ['Welcome to our hotel booking service! How can I assist you today?']}, {'id': 2, 'label': 'ask_room_type', 'is_start': False, 'utterances': ['What type of room would you like to book and for how many nights?']}, {'id': 3, 'label': 'confirm_details', 'is_start': False, 'utterances': ["You've selected a double room for two nights. Is this correct?"]}, {'id': 4, 'label': 'payment', 'is_start': False, 'utterances': ['Great! Please provide your payment details to proceed.']}, {'id': 5, 'label': 'completed', 'is_start':

INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/o

Graph: graph_dict={'edges': [{'source': 1, 'target': 2, 'utterances': ["I'd like to schedule a doctor's appointment"]}, {'source': 2, 'target': 3, 'utterances': ['I need an appointment for a general check-up']}, {'source': 3, 'target': 4, 'utterances': ['Next Monday works for me']}, {'source': 4, 'target': 5, 'utterances': ['Yes, that time is perfect']}, {'source': 5, 'target': 6, 'utterances': ["Here's my insurance information"]}, {'source': 6, 'target': 2, 'utterances': ["I'd like to schedule another appointment"]}], 'nodes': [{'id': 1, 'label': 'welcome', 'is_start': True, 'utterances': ['Welcome to the clinic! How can I assist you today?']}, {'id': 2, 'label': 'ask_appointment_type', 'is_start': False, 'utterances': ['What type of appointment do you need?']}, {'id': 3, 'label': 'ask_date', 'is_start': False, 'utterances': ['When would you like to schedule your appointment?']}, {'id': 4, 'label': 'confirm_time', 'is_start': False, 'utterances': ['We have an opening next Monday at 10

INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/o

...

Graph: graph_dict={'edges': [{'source': 1, 'target': 2, 'utterances': ["Hi, I'm interested in upgrading my mobile phone plan"]}, {'source': 2, 'target': 3, 'utterances': ["I'd like more data, please"]}, {'source': 3, 'target': 4, 'utterances': ['Yes, that sounds good']}, {'source': 4, 'target': 5, 'utterances': ["Here's my payment information"]}, {'source': 5, 'target': 2, 'utterances': ["I'd like to explore more upgrade options"]}], 'nodes': [{'id': 1, 'label': 'welcome', 'is_start': True, 'utterances': ['Welcome! How can I assist you with your mobile plan today?']}, {'id': 2, 'label': 'ask_upgrade_type', 'is_start': False, 'utterances': ['What kind of upgrade are you interested in?']}, {'id': 3, 'label': 'confirm_upgrade', 'is_start': False, 'utterances': ["You've chosen to add more data to your plan. Is that correct?"]}, {'id': 4, 'label': 'payment', 'is_start': False, 'utterances': ['Great choice! The additional data will be $10 extra per month. Please proceed with payment.']}, {'i

INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/o

Graph: graph_dict={'edges': [{'source': 1, 'target': 2, 'utterances': ["Hi, I'd like to book a house cleaning service"]}, {'source': 2, 'target': 3, 'utterances': ['I need a deep cleaning']}, {'source': 3, 'target': 4, 'utterances': ['Yes, that date works for me']}, {'source': 4, 'target': 5, 'utterances': ["Here's my address"]}, {'source': 5, 'target': 6, 'utterances': ["Yes, I'd like to add window cleaning"]}, {'source': 6, 'target': 7, 'utterances': ["Here's my payment information"]}, {'source': 7, 'target': 2, 'utterances': ["I'd like to book another cleaning service"]}], 'nodes': [{'id': 1, 'label': 'welcome', 'is_start': True, 'utterances': ['Welcome to our cleaning service! How can I assist you today?']}, {'id': 2, 'label': 'ask_service_type', 'is_start': False, 'utterances': ['What type of cleaning service are you interested in?']}, {'id': 3, 'label': 'confirm_date', 'is_start': False, 'utterances': ['We have availability on the 15th. Does that work for you?']}, {'id': 4, 'labe

INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 401 Unauthorized"


AuthenticationError: Error code: 401 - {'error': {'code': '401', 'message': '[BricksLLM] key InnPr********************************************** has been revoked', 'type': ''}}